In [13]:
import pyspark
import s3fs

In [14]:
sc = pyspark.SparkContext.getOrCreate()
ss = pyspark.sql.SparkSession.builder.getOrCreate()

In [15]:
s3_address = "s3a://msds630-kaggle-competition/"
dataset_addr_book = {}
dataset_name_list = ["events", "messages", "attributes", "sessions"]
for name in dataset_name_list:
    dataset_addr_book[name] = "".join([s3_address, name, ".csv"])
    
dataset_addr_book

{'attributes': 's3a://msds630-kaggle-competition/attributes.csv',
 'events': 's3a://msds630-kaggle-competition/events.csv',
 'messages': 's3a://msds630-kaggle-competition/messages.csv',
 'sessions': 's3a://msds630-kaggle-competition/sessions.csv'}

In [16]:
events_rdd = sc.textFile(dataset_addr_book["events"])\
               .map(lambda line : line.encode('ascii', 'ignore'))
cols_events = events_rdd.map(lambda x: x.split(',')).take(1)[0]
print(cols_events)
print(len(cols_events))

['app_id', 'session_id', 'event', 'event_timestamp', 'event_value', 'user_id_hash']
6


In [17]:
events_data = events_rdd.filter(lambda x: 'app_id' not in x)\
                        .map(lambda x: x.split(','))
events_data.take(1)

[['4724682771660800',
  '5558845121177764917',
  '45',
  '1542215397132',
  '0.0',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']]

In [18]:
cols_events_adj = cols_events[1:] # Drop app_id
cols_events_adj.append('is_purchase') # Add column is_Purchase
cols_events_adj

['session_id',
 'event',
 'event_timestamp',
 'event_value',
 'user_id_hash',
 'is_purchase']

In [19]:
events_data_adj = events_data.map(lambda x: x[1:]) # Drop app_id
events_data_adj.take(1)

[['5558845121177764917',
  '45',
  '1542215397132',
  '0.0',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']]

In [20]:
def isPurchase(event):
    if event == '8':
        return '1'
    else:
        return '-1'

# Test
isPurchase('8')
isPurchase('9')

'-1'

In [21]:
# Add is_purchase
events_data_adj = events_data_adj.map(lambda x: [x[0], x[1], x[2], x[3], x[4], isPurchase(x[1])])
events_data_adj.take(1)

[['5558845121177764917',
  '45',
  '1542215397132',
  '0.0',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006',
  '-1']]

In [25]:
# new_events_filename = "events_adj.csv"
with open(new_events_filename, "w") as f:
    f.write(','.join(cols_events_adj))
    f.write('\n')
    for line in events_data_adj.collect():        
        f.write(','.join(line))
        f.write('\n')        

In [12]:
sc.stop()